In [1]:
# Install required packages for MoI PDF scraper
import subprocess
import sys

def install_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Required packages
packages = ['beautifulsoup4', 'requests', 'lxml']

for package in packages:
    try:
        __import__(package.replace('-', '_'))
        print(f"✅ {package} already installed")
    except ImportError:
        print(f"📦 Installing {package}...")
        install_package(package)
        print(f"✅ {package} installed successfully")

print("🎉 All required packages are ready!")

📦 Installing beautifulsoup4...
✅ beautifulsoup4 installed successfully
✅ requests already installed
✅ lxml already installed
🎉 All required packages are ready!


In [2]:
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import time

# Your ScraperAPI Key
SCRAPER_API_KEY = "2ef897558ee673a53e6c66518eac104d"  # Replace with your actual API key

# Target website URL
base_url = "https://misa.gov.sa/ar/activities/laws-regulations/"
scraper_url = f"http://api.scraperapi.com?api_key={SCRAPER_API_KEY}&url={base_url}"

download_folder = "moi_pdfs"
os.makedirs(download_folder, exist_ok=True)

def download_pdf(url, retries=3, delay=5):
    filename = url.split("/")[-1]
    for ch in '<>:"/\\|?*':
        filename = filename.replace(ch, "_")
    filepath = os.path.join(download_folder, filename)

    if os.path.exists(filepath) and os.path.getsize(filepath) > 0:
        print(f"✅ Already exists, skipping: {filename}")
        return True

    print(f"⬇️ Downloading: {filename}")
    for attempt in range(1, retries+1):
        try:
            r = requests.get(url, timeout=60)
            if r.status_code == 200:
                with open(filepath, "wb") as f:
                    f.write(r.content)
                print(f"✅ Downloaded: {filename} ({len(r.content)/1024:.1f} KB)")
                return True
            else:
                print(f"⚠️ Attempt {attempt} status: {r.status_code}")
        except requests.exceptions.RequestException as e:
            print(f"⚠️ Attempt {attempt} error: {e}")
        if attempt < retries:
            print(f"⏳ Waiting {delay}s before retry…")
            time.sleep(delay)
    print(f"❌ Failed to download {filename}")
    return False

def main():
    print(f"📁 Created download folder: {os.path.abspath(download_folder)}")
    print("🌐 Connecting to MoI website via ScraperAPI...")
    try:
        response = requests.get(scraper_url, timeout=60)
        response.raise_for_status()
        print(f"✅ Successfully connected to website (Status: {response.status_code})")
        soup = BeautifulSoup(response.content, "html.parser")

        print("🔍 Searching for PDF links...")
        seen, pdf_links = set(), []
        for a in soup.find_all("a", href=True):
            href = a["href"]
            if href and href.lower().endswith(".pdf"):
                full = urljoin(base_url, href)
                if full not in seen:
                    seen.add(full)
                    pdf_links.append(full)
                    print(f"   Found: {full}")

        print(f"📄 Total PDF links found: {len(pdf_links)}")
    except requests.exceptions.RequestException as e:
        print(f"❌ Error connecting to website: {e}")
        pdf_links = []

    if not pdf_links:
        print("❌ No PDF links to download")
        return

    print(f"\n🚀 Starting download of {len(pdf_links)} PDFs...")
    ok = fail = 0
    for i, url in enumerate(pdf_links, 1):
        print(f"\n📥 Processing {i}/{len(pdf_links)}: {url}")
        if download_pdf(url):
            ok += 1
        else:
            fail += 1
        time.sleep(2)

    print("\n✨ Download Summary:")
    print(f"✅ Successful downloads: {ok}")
    print(f"❌ Failed downloads: {fail}")
    print(f"📁 Files saved in: {os.path.abspath(download_folder)}")
    print("\n🏁 MoI PDF scraping completed!")

if __name__ == "__main__":
    main()

📁 Created download folder: c:\Users\ali-d\Desktop\Crawler Updated\Crawler Notebook Updates\moi_pdfs
🌐 Connecting to MoI website via ScraperAPI...
✅ Successfully connected to website (Status: 200)
🔍 Searching for PDF links...
   Found: https://misa.gov.sa/app/uploads/2025/07/Investor-Guide_12-04.pdf
   Found: https://misa.gov.sa/app/uploads/2025/06/THE-INVESTMENT-LAW-IMPLEMENTING-REGULATIONS.pdf
   Found: https://misa.gov.sa/app/uploads/2024/08/Executive-Summary-of-Updated-Investment-Law-En.pdf
   Found: https://misa.gov.sa/app/uploads/2024/08/Investment-Law.pdf
   Found: https://misa.gov.sa/app/uploads/2025/05/Investor-Guide-12-03en.pdf
   Found: https://misa.gov.sa/app/uploads/2024/12/سياسة-الخصوصية_الاصدار-الثالث_EN.pdf
   Found: https://misa.gov.sa/app/uploads/2025/05/Terms_and_Conditions_03.pdf
   Found: https://misa.gov.sa/app/uploads/2025/04/سياسة-الاستخدام-الآمن.pdf
📄 Total PDF links found: 8

🚀 Starting download of 8 PDFs...

📥 Processing 1/8: https://misa.gov.sa/app/uploads/20

In [6]:
# ABBYY Vantage — one-cell token + skills check
import os, json, requests

# ==== CONFIG (edit these) ====
CLIENT_ID = "0JwmpC-UcmL3s7KUdlukm_-Rla2pkg"
CLIENT_SECRET = "eJWlhYAPWHsL1fWDjtFDRTNivI2vvgTTHW"
TOKEN_URL = "https://vantage-eu.abbyy.com/auth2/connect/token"
SKILLS_URL = "https://vantage-eu.abbyy.com/api/publicapi/v1/skills"
SCOPE = "openid permissions global.wildcard"
# Optional: corporate proxy (uncomment & set if needed)
# os.environ["HTTPS_PROXY"] = "http://proxy.host:port"

def get_token():
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    data = {
        "grant_type": "client_credentials",
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET,
        "scope": SCOPE,
    }
    r = requests.post(TOKEN_URL, data=data, headers=headers, timeout=60)
    try:
        r.raise_for_status()
    except Exception as e:
        print("❌ Token request failed:", e)
        print("Status:", r.status_code, "| Body:", r.text[:500])
        raise
    j = r.json()
    token = j.get("access_token")
    if not token:
        raise RuntimeError(f"Token missing in response: {j}")
    os.environ["ABBYY_TOKEN"] = token
    print(f"✅ Token acquired (expires_in={j.get('expires_in')}s) and saved to os.environ['ABBYY_TOKEN']")
    return token

def call_skills(token):
    r = requests.get(
        SKILLS_URL,
        headers={"Authorization": f"Bearer {token}", "Accept": "application/json"},
        timeout=60,
    )
    ct = r.headers.get("Content-Type", "")
    print("Skills status:", r.status_code)
    if r.status_code == 403:
        print("⚠️  Forbidden (403). This client likely lacks permissions.")
        print("   In Vantage Admin → Public API Client → Manage Roles, assign at least 'Skill User'.")
        print("   Then re-run this cell to fetch a NEW token and retry.")
    if ct.startswith("application/json"):
        try:
            data = r.json()
            # print a compact list (id + name if present)
            if isinstance(data, list):
                preview = [
                    {k: v for k, v in s.items() if k.lower() in ("id", "name", "title")}
                    for s in data
                ]
                print(json.dumps(preview, indent=2)[:2000])
            else:
                print(json.dumps(data, indent=2)[:2000])
        except Exception:
            print("Received JSON content but parsing failed; raw text preview:\n", r.text[:1000])
    else:
        print("Non-JSON response preview:\n", r.text[:1000])

# ---- run both steps ----
token = get_token()
call_skills(token)

✅ Token acquired (expires_in=86400s) and saved to os.environ['ABBYY_TOKEN']
Skills status: 200
[
  {
    "id": "1458486c-6a43-4323-b95b-965a0ba98849",
    "name": "arabic"
  }
]


In [7]:
# ABBYY Vantage Transaction Processing Workflow
import os
import json
import time
import requests
from pathlib import Path

# Configuration
SKILL_ID = "1458486c-6a43-4323-b95b-965a0ba98849"
BASE_URL = "https://vantage-eu.abbyy.com/api/publicapi/v1"
PDF_FOLDER = "moi_pdfs"
OUTPUT_FOLDER = "moi_pdfs_ABBYY_Processed"
POLL_INTERVAL = 30  # seconds between status checks
MAX_POLL_ATTEMPTS = 60  # maximum polling attempts (30 min total)

def create_transaction(pdf_file_path, token):
    """Create a transaction for processing a PDF file."""
    url = f"{BASE_URL}/transactions/launch"
    
    # Prepare the model data
    model_data = {
        "files": [
            {
                "index": 0,
                "imageProcessingOptions": {
                    "autoCrop": "Default",
                    "autoOrientation": "Default"
                },
                "registrationParameters": []
            }
        ],
        "registrationParameters": [],
        "skillParameters": []
    }
    
    headers = {
        "accept": "text/plain",
        "Authorization": f"Bearer {token}"
    }
    
    params = {"skillId": SKILL_ID}
    
    # Prepare multipart form data
    files = {
        'Model': (None, json.dumps(model_data), 'application/json'),
        'Files': (os.path.basename(pdf_file_path), open(pdf_file_path, 'rb'), 'application/pdf')
    }
    
    try:
        print(f"🚀 Creating transaction for: {os.path.basename(pdf_file_path)}")
        response = requests.post(url, headers=headers, params=params, files=files, timeout=60)
        
        # Close the file handle
        files['Files'][1].close()
        
        response.raise_for_status()
        
        # Parse response
        if response.headers.get('Content-Type', '').startswith('application/json'):
            result = response.json()
            transaction_id = result.get('transactionId')
            if transaction_id:
                print(f"✅ Transaction created: {transaction_id}")
                return transaction_id
            else:
                print(f"❌ No transaction ID in response: {result}")
                return None
        else:
            # Some APIs return transaction ID as plain text
            transaction_id = response.text.strip().strip('"')
            if transaction_id and len(transaction_id) > 10:  # Basic validation
                print(f"✅ Transaction created: {transaction_id}")
                return transaction_id
            else:
                print(f"❌ Invalid response: {response.text}")
                return None
            
    except requests.RequestException as e:
        print(f"❌ Error creating transaction for {os.path.basename(pdf_file_path)}: {e}")
        return None
    except Exception as e:
        print(f"❌ Unexpected error: {e}")
        return None

def check_transaction_status(transaction_id, token):
    """Check the status of a transaction."""
    url = f"{BASE_URL}/transactions/{transaction_id}"
    headers = {
        "accept": "text/plain",
        "Authorization": f"Bearer {token}"
    }
    
    try:
        response = requests.get(url, headers=headers, timeout=30)
        response.raise_for_status()
        
        if response.headers.get('Content-Type', '').startswith('application/json'):
            return response.json()
        else:
            # Try to parse as JSON anyway
            try:
                return json.loads(response.text)
            except:
                print(f"❌ Non-JSON response: {response.text[:200]}")
                return None
                
    except requests.RequestException as e:
        print(f"❌ Error checking transaction {transaction_id}: {e}")
        return None

def wait_for_transaction_completion(transaction_id, token):
    """Poll transaction status until completion."""
    print(f"⏳ Waiting for transaction {transaction_id} to complete...")
    
    for attempt in range(MAX_POLL_ATTEMPTS):
        status_data = check_transaction_status(transaction_id, token)
        
        if not status_data:
            print(f"❌ Failed to get status for transaction {transaction_id}")
            return None
            
        status = status_data.get('status', 'Unknown')
        print(f"📊 Transaction {transaction_id} status: {status} (attempt {attempt + 1})")
        
        if status == 'Processed':
            print(f"✅ Transaction {transaction_id} completed successfully!")
            return status_data
        elif status in ['Failed', 'Error', 'Cancelled']:
            print(f"❌ Transaction {transaction_id} failed with status: {status}")
            return status_data
        elif status in ['Processing', 'Queued', 'InProgress', 'Pending']:
            if attempt < MAX_POLL_ATTEMPTS - 1:
                print(f"⏳ Still processing... waiting {POLL_INTERVAL}s")
                time.sleep(POLL_INTERVAL)
            else:
                print(f"⏰ Maximum polling attempts reached for transaction {transaction_id}")
                return status_data
        else:
            print(f"❓ Unknown status '{status}' for transaction {transaction_id}")
            if attempt < MAX_POLL_ATTEMPTS - 1:
                time.sleep(POLL_INTERVAL)
    
    return status_data

def download_result_files(transaction_data, token, output_dir):
    """Download all result files from a completed transaction."""
    transaction_id = transaction_data.get('id')
    documents = transaction_data.get('documents', [])
    
    if not documents:
        print(f"⚠️  No documents found in transaction {transaction_id}")
        return []
    
    downloaded_files = []
    os.makedirs(output_dir, exist_ok=True)
    
    for doc_idx, document in enumerate(documents):
        result_files = document.get('resultFiles', [])
        
        for file_idx, result_file in enumerate(result_files):
            file_id = result_file.get('fileId')
            file_name = result_file.get('fileName', f'result_{doc_idx}_{file_idx}')
            
            if not file_id:
                print(f"⚠️  No file ID for result file: {file_name}")
                continue
                
            # Download URL
            download_url = f"{BASE_URL}/transactions/{transaction_id}/files/{file_id}/download"
            headers = {
                "accept": "application/octet-stream",
                "Authorization": f"Bearer {token}"
            }
            
            try:
                print(f"⬇️  Downloading: {file_name}")
                response = requests.get(download_url, headers=headers, stream=True, timeout=60)
                response.raise_for_status()
                
                # Save file
                output_path = os.path.join(output_dir, file_name)
                with open(output_path, 'wb') as f:
                    for chunk in response.iter_content(chunk_size=8192):
                        f.write(chunk)
                
                print(f"✅ Downloaded: {output_path}")
                downloaded_files.append(output_path)
                
            except requests.RequestException as e:
                print(f"❌ Error downloading {file_name}: {e}")
            except Exception as e:
                print(f"❌ Unexpected error downloading {file_name}: {e}")
    
    return downloaded_files

def process_all_pdfs():
    """Main function to process all PDFs in the NCAR_PDFs folder."""
    # Get token (should already be available from previous cell)
    token = os.environ.get('ABBYY_TOKEN')
    if not token:
        print("❌ No ABBYY token found. Please run the token acquisition cell first.")
        return
    
    # Check if PDF folder exists
    if not os.path.exists(PDF_FOLDER):
        print(f"❌ PDF folder '{PDF_FOLDER}' not found. Please run the NCAR scraper first.")
        return
    
    # Get all PDF files
    pdf_files = [f for f in os.listdir(PDF_FOLDER) if f.lower().endswith('.pdf')]
    
    if not pdf_files:
        print(f"❌ No PDF files found in '{PDF_FOLDER}'")
        return
    
    print(f"📁 Found {len(pdf_files)} PDF files to process")
    
    # Track all transactions
    transactions = []
    
    # Create transactions for all PDFs
    print("\n🚀 Creating transactions...")
    for pdf_file in pdf_files:
        pdf_path = os.path.join(PDF_FOLDER, pdf_file)
        transaction_id = create_transaction(pdf_path, token)
        
        if transaction_id:
            transactions.append({
                'id': transaction_id,
                'pdf_file': pdf_file,
                'status': 'Created'
            })
        
        # Small delay between requests
        time.sleep(1)
    
    if not transactions:
        print("❌ No transactions were created successfully.")
        return
    
    print(f"\n✅ Created {len(transactions)} transactions")
    
    # Wait for all transactions to complete
    print("\n⏳ Waiting for all transactions to complete...")
    completed_transactions = []
    
    for transaction in transactions:
        print(f"\n--- Processing transaction for {transaction['pdf_file']} ---")
        result = wait_for_transaction_completion(transaction['id'], token)
        
        if result and result.get('status') == 'Processed':
            completed_transactions.append(result)
            transaction['status'] = 'Completed'
        else:
            transaction['status'] = 'Failed'
    
    # Download all result files
    if completed_transactions:
        print(f"\n⬇️  Downloading results from {len(completed_transactions)} completed transactions...")
        all_downloaded_files = []
        
        for transaction_data in completed_transactions:
            downloaded = download_result_files(transaction_data, token, OUTPUT_FOLDER)
            all_downloaded_files.extend(downloaded)
        
        print(f"\n🎉 Processing complete!")
        print(f"📊 Transactions created: {len(transactions)}")
        print(f"✅ Transactions completed: {len(completed_transactions)}")
        print(f"📥 Files downloaded: {len(all_downloaded_files)}")
        print(f"📁 Output folder: {OUTPUT_FOLDER}")
        
        # Summary
        print(f"\n📋 Transaction Summary:")
        for transaction in transactions:
            status_emoji = "✅" if transaction['status'] == 'Completed' else "❌" if transaction['status'] == 'Failed' else "⏳"
            print(f"  {status_emoji} {transaction['pdf_file']}: {transaction['status']}")
        
    else:
        print("❌ No transactions completed successfully.")

# Run the complete workflow
print("🌟 Starting ABBYY Vantage processing workflow...")
process_all_pdfs()

🌟 Starting ABBYY Vantage processing workflow...
📁 Found 8 PDF files to process

🚀 Creating transactions...
🚀 Creating transaction for: Executive-Summary-of-Updated-Investment-Law-En.pdf
✅ Transaction created: bf9ab72e-9b75-4978-835e-8b2650461d9b
🚀 Creating transaction for: Investment-Law.pdf
✅ Transaction created: 83203f35-ca99-4245-bbad-aff1158322f2
🚀 Creating transaction for: Investor-Guide-12-03en.pdf
✅ Transaction created: 4cdb8231-bbb1-4652-9c37-11e5b890235e
🚀 Creating transaction for: Investor-Guide_12-04.pdf
✅ Transaction created: 43f7a624-296b-49b6-a6f8-39f29c377ec2
🚀 Creating transaction for: Terms_and_Conditions_03.pdf
✅ Transaction created: 531673fa-e62b-4153-bf65-5ceff461f9fe
🚀 Creating transaction for: THE-INVESTMENT-LAW-IMPLEMENTING-REGULATIONS.pdf
✅ Transaction created: 30377a2e-c2d5-4ae6-8a7d-c5e6ab00e48b
🚀 Creating transaction for: سياسة-الاستخدام-الآمن.pdf
✅ Transaction created: bca62e51-5c90-4997-a8e3-ad6fd2f0f078
🚀 Creating transaction for: سياسة-الخصوصية_الاصدار-ال